## XGBoost Modeling

In [0]:
import pandas as pd
import numpy as np

In [0]:
# =====================================================
# STEP 1: Load and Prepare Base Data
# =====================================================
data_path = "/Users/pju307/retail-forecasting/data/sales_data.csv"
df = pd.read_csv(data_path, parse_dates=['Date'])

# Create unique ID (Store + Product)
df["unique_id"] = df["Store ID"].astype(str) + "_" + df["Product ID"].astype(str)

# Sort data chronologically per unique_id
df = df.sort_values(["unique_id", "Date"]).reset_index(drop=True)

In [0]:
# =====================================================
# STEP 2: Time-Based Features
# =====================================================
df["dayofweek"] = df["Date"].dt.dayofweek
df["month"] = df["Date"].dt.month
df["is_weekend"] = df["dayofweek"].isin([5, 6]).astype(int)

In [0]:
df

In [0]:
# =====================================================
# STEP 3: Lag Features (Demand memory)
# =====================================================
for lag in [1, 7, 14, 28]:
    df[f"lag_{lag}"] = df.groupby("unique_id")["Demand"].shift(lag)


In [0]:
# =====================================================
# STEP 4: Rolling Window Statistics
# =====================================================
for window in [7, 14, 28]:
    df[f"rolling_mean_{window}"] = (
        df.groupby("unique_id")["Demand"].shift(1).rolling(window).mean()
    )
    df[f"rolling_std_{window}"] = (
        df.groupby("unique_id")["Demand"].shift(1).rolling(window).std()
    )

In [0]:
# =====================================================
# STEP 5: Context / Identity Features
# =====================================================
one_hot_cols = []
for col in ["Weather Condition", "Category", "Region", "Seasonality"]:
    if col in df.columns:
        one_hot_cols.append(col)

# Perform one-hot encoding with 0/1 integers
df = pd.get_dummies(df, columns=one_hot_cols, drop_first=True, dtype=int)

In [0]:
df

In [0]:
# =====================================================
# STEP 6: External/Causal Features
# =====================================================
keep_features = [
    "Price", "Discount", "Promotion", "Competitor Pricing",
    "Inventory Level", "Units Sold", "Units Ordered",
    "Weather Condition", "Seasonality", "Epidemic"
]
available_features = [f for f in keep_features if f in df.columns]

# =====================================================
# STEP 7: Drop Raw String Columns (no text in final model)
# =====================================================
drop_cols = ["Store ID", "Product ID", "Category", "Region", "Weather Condition", "Seasonality"]
existing_drop_cols = [col for col in drop_cols if col in df.columns]
df = df.drop(columns=existing_drop_cols)

In [0]:
# =====================================================
# STEP 8: Drop Rows with NaN (from lag/rolling)
# =====================================================
df = df.dropna().reset_index(drop=True)

In [0]:
# =====================================================
# STEP 8: Reorder Columns so unique_id is First
# =====================================================
cols = df.columns.tolist()

# Ensure 'unique_id' is first
if "unique_id" in cols:
    cols.remove("unique_id")
    cols = ["unique_id"] + cols

df = df[cols]

### Modeling

In [0]:
import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import multiprocessing

In [0]:
df = df.sort_values(["unique_id", "Date"]).reset_index(drop=True)

target_col = "Demand"
feature_cols = [col for col in df.columns if col not in ["unique_id", "Date", target_col]]

In [0]:
def temporal_train_test_split(df, horizon=7):
    train_df = df.groupby("unique_id").apply(lambda x: x.iloc[:-horizon]).reset_index(drop=True)
    test_df = df.groupby("unique_id").apply(lambda x: x.iloc[-horizon:]).reset_index(drop=True)
    return train_df, test_df

train_df, test_df = temporal_train_test_split(df, horizon=7)

# Convert to XGBoost DMatrix (faster, memory efficient)
dtrain = xgb.DMatrix(train_df[feature_cols], label=train_df[target_col])
dvalid = xgb.DMatrix(test_df[feature_cols], label=test_df[target_col])


In [0]:
def evaluate_forecast(y_true, y_pred, label="Validation"):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((y_true - y_pred) / np.where(y_true == 0, 1e-10, y_true))) * 100
    bias = np.mean(y_pred - y_true)
    r2 = r2_score(y_true, y_pred)

    print(f"\n📊 {label} Metrics")
    print(f"MAE  = {mae:.2f}")
    print(f"MSE  = {mse:.2f}")
    print(f"RMSE = {rmse:.2f}")
    print(f"MAPE = {mape:.2f}%")
    print(f"R²   = {r2:.4f}")
    print(f"Bias = {bias:.2f}")

    return {"MAE": mae, "MSE": mse, "RMSE": rmse, "MAPE": mape, "R2": r2, "Bias": bias}


In [0]:
def objective(trial):
    params = {
        "objective": "reg:squarederror",
        "tree_method": "hist",             # fast CPU histogram algorithm
        "nthread": multiprocessing.cpu_count(),  # utilize all cores
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 4, 12),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "lambda": trial.suggest_float("lambda", 1e-3, 10.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-3, 10.0, log=True),
        "gamma": trial.suggest_float("gamma", 0, 5.0)
    }

    model = xgb.train(
        params,
        dtrain,
        num_boost_round=500,
        evals=[(dtrain, "train"), (dvalid, "valid")],
        early_stopping_rounds=50,
        verbose_eval=False
    )

    preds = model.predict(dvalid)
    rmse = np.sqrt(mean_squared_error(test_df[target_col], preds))
    return rmse

# Run Optuna Study
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30, n_jobs=multiprocessing.cpu_count())

print("\n✅ Best Hyperparameters:")
print(study.best_params)


In [0]:
best_params = study.best_params
best_params.update({
    "objective": "reg:squarederror",
    "tree_method": "hist",
    "nthread": multiprocessing.cpu_count(),
})

model = xgb.train(
    best_params,
    dtrain,
    num_boost_round=1000,
    evals=[(dtrain, "train"), (dvalid, "valid")],
    early_stopping_rounds=75,
    verbose_eval=100
)

In [0]:
train_preds = model.predict(dtrain)
val_preds = model.predict(dvalid)

train_metrics = evaluate_forecast(train_df[target_col], train_preds, "Training")
val_metrics = evaluate_forecast(test_df[target_col], val_preds, "Validation")

metrics_df = pd.DataFrame([train_metrics, val_metrics], index=["Training", "Validation"])
display(metrics_df)

In [0]:
plt.figure(figsize=(15, 8))
plt.suptitle("📈 XGBoost Forecasting Dashboard", fontsize=16, weight="bold")

# Training
plt.subplot(2, 2, 1)
plt.plot(train_df["Date"], train_df[target_col], label="Train Actual", color="steelblue")
plt.plot(train_df["Date"], train_preds, label="Train Predicted", color="orange", linestyle="--")
plt.title("Training: Actual vs Forecast")
plt.legend()

# Validation
plt.subplot(2, 2, 2)
plt.plot(test_df["Date"], test_df[target_col], label="Validation Actual", color="steelblue")
plt.plot(test_df["Date"], val_preds, label="Validation Predicted", color="orange", linestyle="--")
plt.title("Validation: Actual vs Forecast")
plt.legend()

# Residuals
plt.subplot(2, 1, 2)
residuals = train_df[target_col] - train_preds
plt.plot(train_df["Date"], residuals, color="green")
plt.axhline(0, color="black", linestyle="--")
plt.title("Training Residuals")
plt.xlabel("Date")

plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()


## LSTM Modeling